In [41]:
import pandas as pd
import numpy as np

employees = pd.DataFrame({
    'id': list(range(1, 41)),
    'first_name': ['Ana', 'Bob', 'Cara', 'Dave', 'Eve', 'Frank', 'Grace', 'Hank', 'Iris', 'Jack',
                   'Kevin', 'Lisa', 'Mike', 'Nina', 'Omar', 'Paula', 'Quinn', 'Rosa', 'Sam', 'Tina',
                   'Uma', 'Victor', 'Wendy', 'Xavier', 'Yara', 'Zack', 'Amy', 'Brian', 'Chloe', 'Dan',
                   'Ella', 'Fred', 'Gina', 'Hugo', 'Ivy', 'Jake', 'Kim', 'Leo', 'Mia', 'Noah'],
    'last_name': ['Smith', 'Jones', 'Brown', 'Wilson', 'Davis', 'Miller', 'Moore', 'Taylor', 'Anderson', 'Thomas',
                  'Martin', 'Garcia', 'Rodriguez', 'Lee', 'White', 'Harris', 'Clark', 'Lewis', 'Robinson', 'Walker',
                  'Hall', 'Allen', 'Young', 'King', 'Wright', 'Lopez', 'Hill', 'Scott', 'Green', 'Adams',
                  'Baker', 'Nelson', 'Carter', 'Mitchell', 'Perez', 'Roberts', 'Turner', 'Phillips', 'Campbell', 'Parker'],
    'dept': ['Tech', 'Sales', 'Tech', 'Marketing', 'Tech', 'Sales', 'Marketing', 'Tech', 'Sales', None,
             'Tech', 'Sales', 'Marketing', 'Tech', 'HR', 'Sales', 'Tech', 'Marketing', 'Sales', 'Tech',
             'Marketing', 'Tech', 'Sales', 'HR', 'Tech', 'Sales', 'Marketing', 'Tech', 'Sales', 'Marketing',
             'Tech', 'HR', 'Sales', 'Tech', 'Marketing', 'Sales', 'Tech', 'Marketing', 'HR', 'Tech'],
    'salary': [95000, 80000, 92000, 75000, 98000, 82000, 77000, 91000, 79000, None,
               88000, 76000, 73000, 105000, 65000, 84000, 96000, 78000, 81000, 93000,
               74000, 102000, 77000, 68000, 89000, 85000, 71000, 94000, 83000, 72000,
               99000, 67000, 86000, 107000, 76000, 78000, 90000, 79000, 70000, 87000],
    'manager_id': [None, 1, 1, 2, 1, 2, 4, 3, 6, 7,
                   3, 6, 4, 1, None, 2, 3, 4, 6, 1,
                   4, 3, 2, 15, 14, 6, 13, 14, 6, 13,
                   14, 15, 2, 14, 13, 6, 14, 13, 15, 14],
    'hire_date': ['2018-01-15', '2019-03-22', '2018-06-30', '2020-01-10', '2017-09-01', 
                  '2019-11-15', '2020-08-20', '2018-04-12', '2021-02-28', '2022-05-16',
                  '2019-04-01', '2020-11-30', '2021-06-15', '2017-11-20', '2019-07-08',
                  '2020-03-25', '2018-09-10', '2021-01-18', '2019-12-05', '2018-02-28',
                  '2020-09-14', '2017-12-01', '2020-06-22', '2019-08-30', '2018-07-25',
                  '2021-03-10', '2020-12-07', '2018-05-20', '2021-04-12', '2020-10-05',
                  '2017-10-15', '2019-09-18', '2021-05-25', '2018-03-08', '2020-07-13',
                  '2021-08-02', '2018-08-17', '2020-04-27', '2019-10-22', '2018-11-30']
})

departments = pd.DataFrame({
    'dname': ['Tech', 'Sales', 'Marketing', 'HR'],
    'budget': [1400000, 950000, 750000, 280000]
})

In [2]:
## Exercise 71: Salary Compression Analysis
```sql
-- For each department, calculate the "salary compression ratio"
-- (difference between max and min salary) / average salary
-- Also show how many employees are within 10% of the dept average
-- Order by compression ratio (lowest = most compressed salaries)

SyntaxError: invalid syntax (3646898866.py, line 2)

In [55]:
emp = employees.copy()
emp = emp[emp['salary'].notna()]

summary = emp.groupby('dept')['salary'].agg(max='max', min='min', avg='mean')
emp = emp.merge(summary, on='dept')

emp['within10'] = np.where((emp['salary']>=0.9*emp['avg']) & (emp['salary']<=1.1*emp['avg']), 1, 0)
summary['within10']= emp.groupby('dept')['within10'].agg('sum')
summary['compression_ratio'] = (summary['max'] - summary['min']) / summary['avg']
summary['compression_ratio'] = np.where(summary['avg']==0, np.nan, summary['compression_ratio'])
summary


,max,min,avg,within10,compression_ratio
dept,,,,,
HR,70000.0,65000.0,67500.000000,4,0.074074
Marketing,79000.0,71000.0,75000.000000,9,0.106667
Sales,86000.0,76000.0,81000.000000,11,0.123457
Tech,107000.0,87000.0,95066.666667,13,0.210379


In [56]:
## Exercise 72: Consecutive Hire Analysis  
```sql
-- Find all cases where a department hired 2+ people within 90 days
-- Show: dept, first_hire_date, second_hire_date, days_between
-- Include the names of both employees hired

SyntaxError: invalid syntax (1385291989.py, line 2)

In [127]:
# copy, delete empty departments, sort by both dept and hire-date
emp = employees.copy()
emp = emp[emp['dept'].notna()]
emp['hire_date'] = pd.to_datetime(emp['hire_date']) 
emp = emp.sort_values(['dept', 'hire_date'])
# save hire date shifted and id of the shifted column to the current column for comparison
emp[['hire_date_shifted', 'id_shifted', 'first_name_shifted', 'last_name_shifted']] = (emp.groupby('dept')[['hire_date', 'id', 'first_name', 'last_name']].shift(1))

# calculate hiring date difference between current and earlier row(already sorted by hire_Date)
emp['hire_date_diff'] = (emp['hire_date'] - emp['hire_date_shifted']).dt.days

# filter for less than 90 days
emp = emp[emp['hire_date_diff'] <= 90]

emp = emp.rename(columns={'hire_date':'second_hire_date','hire_date_shifted':'first_hire_date','hire_date_diff':'days_between','first_name':'second_hire_first_name', 'last_name':'second_hire_last_name', 'first_name_shifted':'first_hire_first_name', 'last_name_shifted':'first_hire_last_name'})
emp['first_employee'] = emp['first_hire_first_name'] + ' ' + emp['first_hire_last_name']
emp['second_employee'] = emp['second_hire_first_name'] + ' ' + emp['second_hire_last_name']
emp = emp[['first_employee', 'dept','first_hire_date', 'second_hire_date', 'second_employee', 'days_between']]
emp = emp.sort_values(['dept', 'first_hire_date'])
emp

,first_employee,dept,first_hire_date,second_hire_date,second_employee,days_between
23,Omar White,HR,2019-07-08,2019-08-30,Xavier King,53.0
31,Xavier King,HR,2019-08-30,2019-09-18,Fred Nelson,19.0
38,Fred Nelson,HR,2019-09-18,2019-10-22,Mia Campbell,34.0
34,Leo Phillips,Marketing,2020-04-27,2020-07-13,Ivy Perez,77.0
6,Ivy Perez,Marketing,2020-07-13,2020-08-20,Grace Moore,38.0
20,Grace Moore,Marketing,2020-08-20,2020-09-14,Uma Hall,25.0
29,Uma Hall,Marketing,2020-09-14,2020-10-05,Dan Adams,21.0
26,Dan Adams,Marketing,2020-10-05,2020-12-07,Amy Hill,63.0
17,Amy Hill,Marketing,2020-12-07,2021-01-18,Rosa Lewis,42.0
18,Frank Miller,Sales,2019-11-15,2019-12-05,Sam Robinson,20.0


In [140]:
## Exercise 73: Manager Span & Depth
```sql
-- For each manager, calculate:
-- 1. Direct reports count
-- 2. Skip-level reports count (reports of their direct reports)
-- 3. Total reports (direct + skip-level only, max 2 levels)
-- 4. Total salary responsibility (sum of direct and skip-level reports' salaries)
-- Only show managers with at least 1 direct report

SyntaxError: unterminated string literal (detected at line 7) (304618253.py, line 7)